# 🍎 Chat Completions with AIProjectClient 🍏

En este cuaderno, demostraremos cómo realizar **Chat Completions** utilizando el SDK **Azure AI Foundry**. Combinaremos los paquetes **`azure-ai-projects`** y **`azure-ai-inference`** para:

1. **Inicializar** un `AIProjectClient`.
2. **Obtener** un cliente de Chat Completions para realizar llamadas directas al LLM.
3. **Utilizar** una **plantilla de prompt** para añadir el contexto del sistema.
4. **Enviar** mensajes de usuario con una temática de salud y fitness.

## 🏋️ Disclaimer
> **Este ejemplo es solo para demostración y no proporciona asesoramiento real.** 

## 1. Initial Setup

Cargar variables de entorno, crear un `AIProjectClient`, y obtener un `ChatCompletionsClient`. También definiremos un prompt template para mostrar cómo se podría estructurar un system message.


In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path
from azure.identity import DefaultAzureCredential

from azure.ai.projects import AIProjectClient
from azure.ai.inference.models import UserMessage, SystemMessage  # for chat messages

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '../.env')

# Retrieve from environment
connection_string = os.environ.get("PROJECT_CONNECTION_STRING")
model_deployment = 'gpt-4o'

try:
    # Create the project client
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=connection_string,
    )
    print("✅ Successfully created AIProjectClient")
except Exception as e:
    print("❌ Error initializing client:", e)


✅ Successfully created AIProjectClient


### Prompt Template

Definiremos rápidamente un **system message** que establece el contexto como un asistente de fitness amigable que provee un descargo de responsabilidad.

```txt
SYSTEM PROMPT (template):
Eres Virtual GPT, un asesor virtual amigable especializado en recomendar cursos virtuales para Bancolombia.
Recuerda siempre a los usuarios: No soy un asesor académico oficial.
Proporciona recomendaciones claras de cursos, explica brevemente cada uno y anima a los usuarios a explorar oportunidades de aprendizaje virtual.
...
```

Luego pasaremos el contenido del **user** como un mensaje de usuario.


In [5]:
# We'll define a function that runs chat completions with a system prompt & user prompt
def chat_with_assistant(user_input: str):
    """Use chat completions to get a response from our LLM, with system instructions."""
    # Our system message template
    system_text = (
        "Eres Virtual GPT, un asesor virtual amigable especializado en recomendar cursos virtuales.\n"
        "Recuerda siempre a los usuarios: No soy un asesor académico oficial.\n"
        "Proporciona recomendaciones claras de cursos, explica brevemente cada uno y anima a los usuarios a explorar oportunidades de aprendizaje virtual."
    )

    # We'll open the chat completions client
    with project_client.inference.get_chat_completions_client() as chat_client:
        # Construct messages: system + user
        system_message = SystemMessage(content=system_text)
        user_message = UserMessage(content=user_input)

        # Send the request
        response = chat_client.complete(
            model=model_deployment,
            messages=[system_message, user_message]
        )

        return response.choices[0].message.content  # simplest approach: get top choice's content

print("Defined a helper function to do chat completions.")

Defined a helper function to do chat completions.


## 2. Try Chat Completions 🎉

Llamaremos a la función con una pregunta del usuario sobre educación, y veremos el resultado. ¡Siéntete libre de modificar la pregunta o ejecutarla varias veces!


In [ ]:
user_question = "¿Qué cursos ecológicos me recomendarías para comenzar a aprender sobre sostenibilidad y cuidado del medio ambiente?"
reply = chat_with_assistant(user_question)
print("🗣️ User:", user_question)
print("🤖 Assistant:", reply)

## 3. Another Example: Prompt Template with Fill-Ins 📝

Podemos ir un poco más allá y añadir placeholders en el mensaje del sistema. Por ejemplo, imagina que tenemos un userName o goal. Mostraremos un ejemplo mínimo.


In [ ]:
def chat_with_template(user_input: str, user_name: str, goal: str):
    # Construct a system template with placeholders
    system_template = (
        "Eres Virtual GPT, un asesor virtual amigable especializado en recomendar cursos virtuales para SENA.\n"
        "Recuerda siempre a los usuarios: No soy un asesor académico oficial.\n"
        "Recomienda cursos virtuales dirigidos a {name} para alcanzar: {goal}.\n"
        "Proporciona recomendaciones claras de cursos, explica brevemente cada uno y anima a los usuarios a explorar oportunidades de aprendizaje virtual."
    )

    # Fill in placeholders
    system_prompt = system_template.format(name=user_name, goal=goal)

    with project_client.inference.get_chat_completions_client() as chat_client:
        system_msg = SystemMessage(content=system_prompt)
        user_msg = UserMessage(content=user_input)

        response = chat_client.complete(
            model=model_deployment,
            messages=[system_msg, user_msg]
        )

    return response.choices[0].message.content

# Let's try it out
templated_user_input = "¿Qué cursos ecológicos me recomendarías para comenzar a aprender sobre sostenibilidad y cuidado del medio ambiente?"
assistant_reply = chat_with_template(
    templated_user_input,
    user_name="Pablo",
    goal="ser emprendedor en sostenibilidad y medio ambiente"
)
print("🗣️ User:", templated_user_input)
print("🤖 Assistant:", assistant_reply)

🗣️ User: ¿Qué cursos ecológicos me recomendarías para comenzar a aprender sobre sostenibilidad y cuidado del medio ambiente?
🤖 Assistant: ¡Hola, Pablo! ¡Qué gran objetivo te has puesto! La sostenibilidad y el cuidado del medio ambiente son áreas cruciales para nuestro futuro. Aquí tienes algunas recomendaciones de cursos virtuales ofrecidos por el SENA que te ayudarán a comenzar tu camino como emprendedor en sostenibilidad y medio ambiente:

1. **Gestión Ambiental**: Este curso te enseñará los principios básicos de la gestión ambiental, incluyendo temas como la normatividad ambiental, la gestión de residuos sólidos, la calidad del aire y del agua, y la implementación de sistemas de gestión ambiental en empresas y organizaciones.

2. **Emprendimiento Sostenible**: En este curso aprenderás sobre la creación y gestión de proyectos empresariales sostenibles, cómo identificar oportunidades de negocio verde y desarrollar modelos de negocio que generen impacto positivo en el medio ambiente.

